## Explore Toronto

### Part 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Scrape the website for the table and assign the rows from the table
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc = requests.get(url).text
soup = BeautifulSoup(html_doc, 'html.parser')
html_table = soup.find('table')
table_rows = html_table.find_all('tr')

# Add the rows into a dataframe
l = []
for tr in table_rows:  
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
toronto = pd.DataFrame(l, columns=["PostalCode", "Borough","Neighborhood"])

# Remove unwanted rows and characters
toronto = toronto[toronto.Borough != 'Not assigned']
toronto.drop([0], axis=0, inplace=True)
toronto['Neighborhood'] = toronto['Neighborhood'].str.rstrip('\n')

# Group rows by postal code and aggregate the neighborhoods
toronto = toronto.groupby(['PostalCode', 'Borough']).agg([('neighborhood', ', '.join)])
toronto.reset_index(inplace=True)
toronto.columns = ['PostalCode', 'Borough', 'Neighborhood']

# Rename neighborhoods that are "not assigned" with the name of the borough
import numpy as np
toronto['Neighborhood'] = np.where(toronto['Neighborhood'] == 'Not assigned', toronto['Borough'], toronto['Neighborhood'])
toronto.loc[85]

# Have a look at the resulting dataframe
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Part 2

In [ ]:
#!pip install wget

In [3]:
import wget
wget.download('http://cocl.us/Geospatial_data')
coords = pd.read_csv("Geospatial_Coordinates.csv", delimiter=",")
coords.set_index('Postal Code', inplace=True)
toronto.set_index('PostalCode', inplace=True)
toronto['Latitude'] = coords['Latitude']
toronto['Longitude'] = coords['Longitude']
toronto.reset_index(inplace=True)
toronto.head()

100% [................................................................................] 2891 / 2891

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 3

**Install and import libraries**

In [4]:
#!pip install folium

In [5]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize
import folium

**Create a map of Toronto**

In [7]:
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

**Create map for the boroughs containing "Toronto"**

In [25]:
toronto_data = toronto[toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
btoronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(btoronto)  
    
btoronto

**Define function to get up to 100 venues within a 500m radius of each neighborhood and run on boroughs containing "Toronto"**

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [38]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15


**Convert the JSON structure into a data frame**

In [41]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")	# one hot encoding
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 	# add neighborhood column back to dataframe
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])	# move neighborhood column to the first column
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0


In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

**Return a dataframe with the top 5 most common venues for each neighborhood**

In [68]:
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(15)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Grocery Store,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Recording Studio,Auto Workshop,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Bakery,Market
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Chinese Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Dumpling Restaurant,Coffee Shop
8,Christie,Grocery Store,Café,Park,Restaurant,Nightclub
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Restaurant


In [69]:
# Run k-means to cluster the neighbourhood into 5 clusters
kclusters = 5	# set number of clusters
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)	# run k-means clustering
kmeans.labels_[0:20] 	# check cluster labels generated for each row in the dataframe

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1])

**Create a new dataframe that includes the cluster as well as the top five venues for each neighborhood**

In [70]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)	# add clustering labels
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')	# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Music Venue,Wings Joint,Department Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Park,Sandwich Place,Coffee Shop,Food & Drink Shop,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Bus Line,Park,Construction & Landscaping,Swim School,Wings Joint


**Visualise the clusters on a map**

In [71]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)	# create map

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Clusters that are not cluster 1 do not contain any coffee shops, they have non food related venues as their most common venue, so are most likely not as central as cluster 1. From the map, we can see that these clusters are further from central Toronto**

In [76]:
toronto_merged[toronto_merged['Cluster Labels'] != 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Playground,Trail,Tennis Court,Wings Joint,Dog Run
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Park,Playground,Trail,Wings Joint,Department Store
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Garden,Wings Joint,Department Store,Event Space,Ethiopian Restaurant
23,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,3,Bus Line,Trail,Jewelry Store,Sushi Restaurant,Wings Joint


**Cluster 1 contains nearly all of the venues. All of these venues contain cafes or coffee shops in the top 5 most common venue**

In [77]:
toronto_merged[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Music Venue,Wings Joint,Department Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Park,Sandwich Place,Coffee Shop,Food & Drink Shop,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Bus Line,Park,Construction & Landscaping,Swim School,Wings Joint
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Park,Clothing Store,Sandwich Place,Food & Drink Shop,Hotel
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Gym / Fitness Center
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,1,Coffee Shop,Pub,Light Rail Station,Convenience Store,Sushi Restaurant
11,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,Coffee Shop,Restaurant,Pub,Bakery,Market
